<!-- HTML file automatically generated from DocOnce source (https://github.com/doconce/doconce/)
doconce format html week13.do.txt --no_mako -->
<!-- dom:TITLE: Advanced machine learning and data analysis for the physical sciences -->

# Advanced machine learning and data analysis for the physical sciences
**Morten Hjorth-Jensen**, Department of Physics and Center for Computing in Science Education, University of Oslo, Norway and Department of Physics and Astronomy and Facility for Rare Isotope Beams, Michigan State University, East Lansing, Michigan, USA

Date: **April 16, 2024**

## Plans for the week April 15-19, 2024

**Deep generative models.**

1. Finalizing discussion of Boltzmann machines, implementations using TensorFlow and Pytorch

2. Discussion of other energy-based models and Langevin sampling

3. Variational Autoencoders (VAE), mathematics

4. [Video of lecture](https://youtu.be/rw-NBN293o4)

5. [Whiteboard notes](https://github.com/CompPhysics/AdvancedMachineLearning/blob/main/doc/HandwrittenNotes/2024/NotesApril16.pdf)

## Readings
1. Reading recommendation: Goodfellow et al, for VAEs see sections 20.10-20.11

2. To create Boltzmann machine using Keras, see Babcock and Bali chapter 4, see <https://github.com/PacktPublishing/Hands-On-Generative-AI-with-Python-and-TensorFlow-2/blob/master/Chapter_4/models/rbm.py>

3. See Foster, chapter 7 on energy-based models at <https://github.com/davidADSP/Generative_Deep_Learning_2nd_Edition/tree/main/notebooks/07_ebm/01_ebm>

<!-- todo: add about Langevin sampling, see <https://www.lyndonduong.com/sgmcmc/> -->
<!-- code for VAEs applied to MNIST and CIFAR perhaps -->

## Reminder from last week and layout of lecture this week

1. We will present first a reminder from last week, see for example the jupyter-notebook at <https://github.com/CompPhysics/AdvancedMachineLearning/blob/main/doc/pub/week12/ipynb/week12.ipynb>

2. We will then discuss codes as well as other energy-based models and Langevin sampling instead of Gibbs or Metropolis sampling.

3. Thereafter we start our discussions of Variational autoencoders and Generalized adversarial networks

## Code for RBMs using PyTorch for a binary-binary RBM

In [1]:
%matplotlib inline

import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchvision.utils import make_grid , save_image
import matplotlib.pyplot as plt


batch_size = 64
train_loader = torch.utils.data.DataLoader(
datasets.MNIST('./data',
    train=True,
    download = True,
    transform = transforms.Compose(
        [transforms.ToTensor()])
     ),
     batch_size=batch_size
)

test_loader = torch.utils.data.DataLoader(
datasets.MNIST('./data',
    train=False,
    transform=transforms.Compose(
    [transforms.ToTensor()])
    ),
    batch_size=batch_size)


class RBM(nn.Module):
   def __init__(self,
               n_vis=784,
               n_hin=500,
               k=5):
        super(RBM, self).__init__()
        self.W = nn.Parameter(torch.randn(n_hin,n_vis)*1e-2)
        self.v_bias = nn.Parameter(torch.zeros(n_vis))
        self.h_bias = nn.Parameter(torch.zeros(n_hin))
        self.k = k
    
   def sample_from_p(self,p):
       return F.relu(torch.sign(p - Variable(torch.rand(p.size()))))
    
   def v_to_h(self,v):
        p_h = F.sigmoid(F.linear(v,self.W,self.h_bias))
        sample_h = self.sample_from_p(p_h)
        return p_h,sample_h
    
   def h_to_v(self,h):
        p_v = F.sigmoid(F.linear(h,self.W.t(),self.v_bias))
        sample_v = self.sample_from_p(p_v)
        return p_v,sample_v
        
   def forward(self,v):
        pre_h1,h1 = self.v_to_h(v)
        
        h_ = h1
        for _ in range(self.k):
            pre_v_,v_ = self.h_to_v(h_)
            pre_h_,h_ = self.v_to_h(v_)
        
        return v,v_
    
   def free_energy(self,v):
        vbias_term = v.mv(self.v_bias)
        wx_b = F.linear(v,self.W,self.h_bias)
        hidden_term = wx_b.exp().add(1).log().sum(1)
        return (-hidden_term - vbias_term).mean()




rbm = RBM(k=1)
train_op = optim.SGD(rbm.parameters(),0.1)

for epoch in range(10):
    loss_ = []
    for _, (data,target) in enumerate(train_loader):
        data = Variable(data.view(-1,784))
        sample_data = data.bernoulli()
        
        v,v1 = rbm(sample_data)
        loss = rbm.free_energy(v) - rbm.free_energy(v1)
        loss_.append(loss.data)
        train_op.zero_grad()
        loss.backward()
        train_op.step()

    print("Training loss for {} epoch: {}".format(epoch, np.mean(loss_)))


def show_adn_save(file_name,img):
    npimg = np.transpose(img.numpy(),(1,2,0))
    f = "./%s.png" % file_name
    plt.imshow(npimg)
    plt.imsave(f,npimg)

show_adn_save("real",make_grid(v.view(32,1,28,28).data))
show_adn_save("generate",make_grid(v1.view(32,1,28,28).data))

## RBM using TensorFlow and Keras

1. To create Boltzmann machine using Keras, see Babcock and Bali chapter 4, see <https://github.com/PacktPublishing/Hands-On-Generative-AI-with-Python-and-TensorFlow-2/blob/master/Chapter_4/models/rbm.py>

## Codes for Energy-based models

See discussions in Foster, chapter 7 on energy-based models at <https://github.com/davidADSP/Generative_Deep_Learning_2nd_Edition/tree/main/notebooks/07_ebm/01_ebm>

That notebook is based on a recent article by Du and Mordatch, **Implicit generation and modeling with energy-based models**, see <https://arxiv.org/pdf/1903.08689.pdf.>

## Langevin sampling

Also called Stochastic gradient Langevin dynamics (SGLD), is sampling
technique composed of characteristics from Stochastic gradient descent
(SGD) and Langevin dynamics, a mathematical extension of the Langevin
equation.  The SGLD is an iterative
optimization algorithm which uses minibatching to create a stochastic
gradient estimator, as used in SGD to optimize a differentiable
objective function.

Unlike traditional SGD, SGLD can be used for
Bayesian learning as a sampling method. SGLD may be viewed as Langevin
dynamics applied to posterior distributions, but the key difference is
that the likelihood gradient terms are minibatched, like in SGD. SGLD,
like Langevin dynamics, produces samples from a posterior distribution
of parameters based on available data.

## More on the SGLD

The SGLD uses the probability $p(\theta)$ (note that we limit
ourselves to just a variable $\theta$) and updates the **log** of this
probability by initializing it through some random prior distribution,
normally just a uniform distribution which takes values between
$\theta\in [-1,1]$,

The update is given by

$$
\theta_{i+1}=\theta_{i}+\eta \nabla_{\theta} \log{p(\theta_{i})}+\sqrt{\eta}w_i,
$$

where $w_i\sim N(0,1)$ are normally distributed with mean zero and
variance one and $i=0,1,\dots,k$, with $k$ the final number of
iterations.  The parameter $\eta$ is the learning rate. The term
$\sqrt{\eta}w_i$ introduces **noise** in the equation.

## Code example of Langevin Sampling

In our calculations the gradient is calculated using the model we have
for the probability distribution. For an energy-based model this gives
us a derivative which involves the so-called positive and negative
phases discussed last week.

Read more about Langevin sampling at for example
<https://www.lyndonduong.com/sgmcmc/>. This site contains a nice
example of a PyTorch code which implements Langevin sampling.

## Theory of Variational Autoencoders

Let us remind ourself about what an autoencoder is, see the jupyter-notebook at <https://github.com/CompPhysics/AdvancedMachineLearning/blob/main/doc/pub/week10/ipynb/week10.ipynb>.

## The Autoencoder again

Autoencoders are neural networks where the outputs are its own
inputs. They are split into an **encoder part**
which maps the input $\boldsymbol{x}$ via a function $f(\boldsymbol{x},\boldsymbol{W})$ (this
is the encoder part) to a **so-called code part** (or intermediate part)
with the result $\boldsymbol{h}$

$$
\boldsymbol{h} = f(\boldsymbol{x},\boldsymbol{W})),
$$

where $\boldsymbol{W}$ are the weights to be determined.  The **decoder** parts maps, via its own parameters (weights given by the matrix $\boldsymbol{V}$ and its own biases) to 
the final ouput

$$
\tilde{\boldsymbol{x}} = g(\boldsymbol{h},\boldsymbol{V})).
$$

The goal is to minimize the construction error, often done by optimizing the means squared error.

## Schematic image of an Autoencoder

<!-- dom:FIGURE: [figures/ae1.png, width=700 frac=1.0] -->
<!-- begin figure -->

<img src="figures/ae1.png" width="700"><p style="font-size: 0.9em"><i>Figure 1: </i></p>
<!-- end figure -->

## Mathematics of Variational Autoencoders

We have defined earlier a probability (marginal) distribution with hidden variables $\boldsymbol{h}$ and parameters $\boldsymbol{\Theta}$ as

$$
p(\boldsymbol{x};\boldsymbol{\Theta}) = \int d\boldsymbol{h}p(\boldsymbol{x},\boldsymbol{h};\boldsymbol{\Theta}),
$$

for continuous variables $\boldsymbol{h}$ and

$$
p(\boldsymbol{x};\boldsymbol{\Theta}) = \sum_{\boldsymbol{h}}p(\boldsymbol{x},\boldsymbol{h};\boldsymbol{\Theta}),
$$

for discrete stochastic events $\boldsymbol{h}$. The variables $\boldsymbol{h}$ are normally called the **latent variables** in the theory of autoencoders. We will also call then for that here.

## Using the conditional probability

Using the the definition of the conditional probabilities $p(\boldsymbol{x}\vert\boldsymbol{h};\boldsymbol{\Theta})$, $p(\boldsymbol{h}\vert\boldsymbol{x};\boldsymbol{\Theta})$ and 
and the prior $p(\boldsymbol{h})$, we can rewrite the above equation as

$$
p(\boldsymbol{x};\boldsymbol{\Theta}) = \sum_{\boldsymbol{h}}p(\boldsymbol{x}\vert\boldsymbol{h};\boldsymbol{\Theta})p(\boldsymbol{h},
$$

which allows us to make the dependence of $\boldsymbol{x}$ on $\boldsymbol{h}$
explicit by using the law of total probability. The intuition behind
this approach for finding the marginal probability for $\boldsymbol{x}$ is to
optimize the above equations with respect to the parameters
$\boldsymbol{\Theta}$.  This is done normally by maximizing the probability,
the so-called maximum-likelihood approach discussed earlier.

## VAEs versus autoencoders

This trained probability is assumed to be able to produce similar
samples as the input.  In VAEs it is then common to compare via for
example the mean-squared error or the cross-entropy the predicted
values with the input values.  Compared with autoencoders, we are now
producing a probability instead of a functions which mimicks the
input.

In VAEs, the choice of this output distribution is often Gaussian,
meaning that the conditional probability is

$$
p(\boldsymbol{x}\vert\boldsymbol{h};\boldsymbol{\Theta})=N(\boldsymbol{x}\vert f(\boldsymbol{h};\boldsymbol{\Theta}), \sigma^2\times \boldsymbol{I}),
$$

with mean value given by the function $f(\boldsymbol{h};\boldsymbol{\Theta})$ and a
diagonal covariance matrix multiplied by a parameter $\sigma^2$ which
is treated as a hyperparameter.

## Gradient descent

By having a Gaussian distribution, we can use gradient descent (or any
other optimization technique) to increase $p(\boldsymbol{x};\boldsymbol{\Theta})$ by
making $f(\boldsymbol{h};\boldsymbol{\Theta})$ approach $\boldsymbol{x}$ for some $\boldsymbol{h}$,
gradually making the training data more likely under the generative
model. The important property is simply that the marginal probability
can be computed, and it is continuous in $\boldsymbol{\Theta}$..

## Are VAEs just modified autoencoders?

The mathematical basis of VAEs actually has relatively little to do
with classical autoencoders, for example the sparse autoencoders or
denoising autoencoders discussed earlier.

VAEs approximately maximize the probability equation discussed
above. They are called autoencoders only because the final training
objective that derives from this setup does have an encoder and a
decoder, and resembles a traditional autoencoder. Unlike sparse
autoencoders, there are generally no tuning parameters analogous to
the sparsity penalties. And unlike sparse and denoising autoencoders,
we can sample directly from $p(\boldsymbol{x})$ without performing Markov
Chain Monte Carlo.

## Training VAEs

To solve the integral or sum for $p(\boldsymbol{x})$, there are two problems
that VAEs must deal with: how to define the latent variables $\boldsymbol{h}$,
that is decide what information they represent, and how to deal with
the integral over $\boldsymbol{h}$.  VAEs give a definite answer to both.

## Kullback-Leibler relative entropy (notation to be updated)

When the goal of the training is to approximate a probability
distribution, as it is in generative modeling, another relevant
measure is the **Kullback-Leibler divergence**, also known as the
relative entropy or Shannon entropy. It is a non-symmetric measure of the
dissimilarity between two probability density functions $p$ and
$q$. If $p$ is the unkown probability which we approximate with $q$,
we can measure the difference by

$$
\begin{align*}
	\text{KL}(p||q) = \int_{-\infty}^{\infty} p (\boldsymbol{x}) \log \frac{p(\boldsymbol{x})}{q(\boldsymbol{x})}  d\boldsymbol{x}.
\end{align*}
$$

## Kullback-Leibler divergence and RBMs

Thus, the Kullback-Leibler divergence between the distribution of the
training data $f(\boldsymbol{x})$ and the model marginal distribution $p(\boldsymbol{x};\boldsymbol{\Theta})$ from an RBM is

$$
\begin{align*}
	\text{KL} (f(\boldsymbol{x})|| p(\boldsymbol{x}| \boldsymbol{\Theta})) =& \int_{-\infty}^{\infty}
	f (\boldsymbol{x}) \log \frac{f(\boldsymbol{x})}{p(\boldsymbol{x}; \boldsymbol{\Theta})} d\boldsymbol{x} \\
	=& \int_{-\infty}^{\infty} f(\boldsymbol{x}) \log f(\boldsymbol{x}) d\boldsymbol{x} - \int_{-\infty}^{\infty} f(\boldsymbol{x}) \log
	p(\boldsymbol{x};\boldsymbol{\Theta}) d\boldsymbol{x} \\
	%=& \mathbb{E}_{f(\boldsymbol{x})} (\log f(\boldsymbol{x})) - \mathbb{E}_{f(\boldsymbol{x})} (\log p(\boldsymbol{x}; \boldsymbol{\Theta}))
	=& \langle \log f(\boldsymbol{x}) \rangle_{f(\boldsymbol{x})} - \langle \log p(\boldsymbol{x};\boldsymbol{\Theta}) \rangle_{f(\boldsymbol{x})} \\
	=& \langle \log f(\boldsymbol{x}) \rangle_{data} + \langle E(\boldsymbol{x}) \rangle_{data} + \log Z.
\end{align*}
$$

## Maximizing log-likelihood

The first term is constant with respect to $\boldsymbol{\Theta}$ since
$f(\boldsymbol{x})$ is independent of $\boldsymbol{\Theta}$. Thus the Kullback-Leibler
divergence is minimal when the second term is minimal. The second term
is the log-likelihood cost function, hence minimizing the
Kullback-Leibler divergence is equivalent to maximizing the
log-likelihood.

## Back to VAEs

We want to train the marginal probability with some latent variables $\boldsymbol{h}$

$$
p(\boldsymbol{x};\boldsymbol{\Theta}) = \int d\boldsymbol{h}p(\boldsymbol{x},\boldsymbol{h};\boldsymbol{\Theta}),
$$

for the continuous version (see previous slides for the discrete variant).

## Using the KL divergence

In practice, for most $\boldsymbol{h}$, $p(\boldsymbol{x}\vert \boldsymbol{h}; \boldsymbol{\Theta})$
will be nearly zero, and hence contribute almost nothing to our
estimate of $p(\boldsymbol{x})$.

The key idea behind the variational autoencoder is to attempt to
sample values of $\boldsymbol{h}$ that are likely to have produced $\boldsymbol{x}$,
and compute $p(\boldsymbol{x})$ just from those.

This means that we need a new function $Q(\boldsymbol{h}|\boldsymbol{x})$ which can
take a value of $\boldsymbol{x}$ and give us a distribution over $\boldsymbol{h}$
values that are likely to produce $\boldsymbol{x}$.  Hopefully the space of
$\boldsymbol{h}$ values that are likely under $Q$ will be much smaller than
the space of all $\boldsymbol{h}$'s that are likely under the prior
$p(\boldsymbol{h})$.  This lets us, for example, compute $E_{\boldsymbol{h}\sim
Q}p(\boldsymbol{x}\vert \boldsymbol{h})$ relatively easily. Note that we drop
$\boldsymbol{\Theta}$ from here and for notational simplicity.

## Kullback-Leibler again

However, if $\boldsymbol{h}$ is sampled from an arbitrary distribution with
PDF $Q(\boldsymbol{h})$, which is not $\mathcal{N}(0,I)$, then how does that
help us optimize $p(\boldsymbol{x})$?

The first thing we need to do is relate
$E_{\boldsymbol{h}\sim Q}P(\boldsymbol{x}\vert \boldsymbol{h})$ and $p(\boldsymbol{x})$.  We will see where $Q$ comes from later.

The relationship between $E_{\boldsymbol{h}\sim Q}p(\boldsymbol{x}\vert \boldsymbol{h})$ and $p(\boldsymbol{x})$ is one of the cornerstones of variational Bayesian methods.
We begin with the definition of Kullback-Leibler divergence (KL divergence or $\mathcal{D}$) between $p(\boldsymbol{h}\vert \boldsymbol{x})$ and $Q(\boldsymbol{h})$, for some arbitrary $Q$ (which may or may not depend on $\boldsymbol{x}$):

$$
\mathcal{D}\left[Q(\boldsymbol{h})\|p(\boldsymbol{h}|\boldsymbol{x})\right]=E_{\boldsymbol{h}\sim Q}\left[\log Q(\boldsymbol{h}) - \log p(\boldsymbol{h}|\boldsymbol{x}) \right].
$$

## And applying Bayes rule

We can get both $p(\boldsymbol{x})$ and $p(\boldsymbol{x}\vert \boldsymbol{h})$ into this equation by applying Bayes rule to $p(\boldsymbol{h}|\boldsymbol{x})$

$$
\mathcal{D}\left[Q(\boldsymbol{h})\|p(\boldsymbol{h}\vert \boldsymbol{x})\right]=E_{\boldsymbol{h}\sim Q}\left[\log Q(\boldsymbol{h}) - \log p(\boldsymbol{x}|\boldsymbol{h}) - \log p(\boldsymbol{h}) \right] + \log p(\boldsymbol{x}).
$$

Here, $\log p(\boldsymbol{x})$ comes out of the expectation because it does not depend on $\boldsymbol{h}$.
Negating both sides, rearranging, and contracting part of $E_{\boldsymbol{h}\sim Q}$ into a KL-divergence terms yields:

$$
\log p(\boldsymbol{x}) - \mathcal{D}\left[Q(\boldsymbol{h})\|p(\boldsymbol{h}\vert \boldsymbol{x})\right]=E_{\boldsymbol{h}\sim Q}\left[\log p(\boldsymbol{x}\vert\boldsymbol{h})  \right] - \mathcal{D}\left[Q(\boldsymbol{h})\|P(\boldsymbol{h})\right].
$$

## Rearranging

Using Bayes rule we obtain

$$
E_{\boldsymbol{h}\sim Q}\left[\log p(y_i|\boldsymbol{h},x_i)\right]=E_{\boldsymbol{h}\sim Q}\left[\log p(\boldsymbol{h}|y_i,x_i) - \log p(\boldsymbol{h}|x_i) + \log p(y_i|x_i) \right]
$$

Rearranging the terms and subtracting $E_{\boldsymbol{h}\sim Q}\log Q(\boldsymbol{h})$ from both sides gives

$$
\begin{array}{c}
\log P(y_i|x_i) - E_{\boldsymbol{h}\sim Q}\left[\log Q(\boldsymbol{h})-\log p(\boldsymbol{h}|x_i,y_i)\right]=\hspace{10em}\\
\hspace{10em}E_{\boldsymbol{h}\sim Q}\left[\log p(y_i|\boldsymbol{h},x_i)+\log p(\boldsymbol{h}|x_i)-\log Q(\boldsymbol{h})\right]
\end{array}
$$

Note that $\boldsymbol{x}$ is fixed, and $Q$ can be \textit{any} distribution, not
just a distribution which does a good job mapping $\boldsymbol{x}$ to the $\boldsymbol{h}$'s
that can produce $X$.

## Inferring the probability

Since we are interested in inferring $p(\boldsymbol{x})$, it makes sense to
construct a $Q$ which \textit{does} depend on $\boldsymbol{x}$, and in particular,
one which makes $\mathcal{D}\left[Q(\boldsymbol{h})\|p(\boldsymbol{h}|\boldsymbol{x})\right]$ small

$$
\log p(\boldsymbol{x}) - \mathcal{D}\left[Q(\boldsymbol{h}|\boldsymbol{x})\|p(\boldsymbol{h}|\boldsymbol{x})\right]=E_{\boldsymbol{h}\sim Q}\left[\log p(\boldsymbol{x}|\boldsymbol{h})  \right] - \mathcal{D}\left[Q(\boldsymbol{h}|\boldsymbol{x})\|p(\boldsymbol{h})\right].
$$

Hence, during training, it makes sense to choose a $Q$ which will make
$E_{\boldsymbol{h}\sim Q}[\log Q(\boldsymbol{h})-$ $\log p(\boldsymbol{h}|x_i,y_i)]$ (a
$\mathcal{D}$-divergence) small, such that the right hand side is a
close approximation to $\log p(y_i|y_i)$.

## Central equation of VAEs

This equation serves as the core of the variational autoencoder, and
it is worth spending some time thinking about what it means.

1. The left hand side has the quantity we want to maximize, namely $\log p(\boldsymbol{x})$ plus an error term.

2. The right hand side is something we can optimize via stochastic gradient descent given the right choice of $Q$.

## Setting up SGD
So how can we perform stochastic gradient descent?

First we need to be a bit more specific about the form that $Q(\boldsymbol{h}|\boldsymbol{x})$
will take.  The usual choice is to say that
$Q(\boldsymbol{h}|\boldsymbol{x})=\mathcal{N}(\boldsymbol{h}|\mu(\boldsymbol{x};\vartheta),\Sigma(;\vartheta))$, where
$\mu$ and $\Sigma$ are arbitrary deterministic functions with
parameters $\vartheta$ that can be learned from data (we will omit
$\vartheta$ in later equations).  In practice, $\mu$ and $\Sigma$ are
again implemented via neural networks, and $\Sigma$ is constrained to
be a diagonal matrix.

## More on the SGD

The name variational "autoencoder" comes from
the fact that $\mu$ and $\Sigma$ are "encoding" $\boldsymbol{x}$ into the latent
space $\boldsymbol{h}$.  The advantages of this choice are computational, as they
make it clear how to compute the right hand side.  The last
term---$\mathcal{D}\left[Q(\boldsymbol{h}|\boldsymbol{x})\|p(\boldsymbol{h})\right]$---is now a KL-divergence
between two multivariate Gaussian distributions, which can be computed
in closed form as:

$$
\begin{array}{c}
 \mathcal{D}[\mathcal{N}(\mu_0,\Sigma_0) \| \mathcal{N}(\mu_1,\Sigma_1)] = \hspace{20em}\\
  \hspace{5em}\frac{ 1 }{ 2 } \left( \mathrm{tr} \left( \Sigma_1^{-1} \Sigma_0 \right) + \left( \mu_1 - \mu_0\right)^\top \Sigma_1^{-1} ( \mu_1 - \mu_0 ) - k + \log \left( \frac{ \det \Sigma_1 }{ \det \Sigma_0  } \right)  \right)
\end{array}
$$

where $k$ is the dimensionality of the distribution.

## Simplification
In our case, this simplifies to:

$$
\begin{array}{c}
 \mathcal{D}[\mathcal{N}(\mu(X),\Sigma(X)) \| \mathcal{N}(0,I)] = \hspace{20em}\\
\hspace{6em}\frac{ 1 }{ 2 } \left( \mathrm{tr} \left( \Sigma(X) \right) + \left( \mu(X)\right)^\top ( \mu(X) ) - k - \log\det\left(  \Sigma(X)  \right)  \right).
\end{array}
$$

## Terms to compute

The first term on the right hand side is a bit more tricky.
We could use sampling to estimate $E_{z\sim Q}\left[\log P(X|z)  \right]$, but getting a good estimate would require passing many samples of $z$ through $f$, which would be expensive.
Hence, as is standard in stochastic gradient descent, we take one sample of $z$ and treat $\log P(X|z)$ for that $z$ as an approximation of $E_{z\sim Q}\left[\log P(X|z)  \right]$.
After all, we are already doing stochastic gradient descent over different values of $X$ sampled from a dataset $D$.
The full equation we want to optimize is:

$$
\begin{array}{c}
    E_{X\sim D}\left[\log P(X) - \mathcal{D}\left[Q(z|X)\|P(z|X)\right]\right]=\hspace{16em}\\
\hspace{10em}E_{X\sim D}\left[E_{z\sim Q}\left[\log P(X|z)  \right] - \mathcal{D}\left[Q(z|X)\|P(z)\right]\right].
\end{array}
$$

## Computing the gradients

If we take the gradient of this equation, the gradient symbol can be moved into the expectations.
Therefore, we can sample a single value of $X$ and a single value of $z$ from the distribution $Q(z|X)$, and compute the gradient of:

<!-- Equation labels as ordinary links -->
<div id="_auto1"></div>

$$
\begin{equation}
 \log P(X|z)-\mathcal{D}\left[Q(z|X)\|P(z)\right].
\label{_auto1} \tag{1}
\end{equation}
$$

We can then average the gradient of this function over arbitrarily many samples of $X$ and $z$, and the result converges to the gradient.

There is, however, a significant problem
$E_{z\sim Q}\left[\log P(X|z)  \right]$ depends not just on the parameters of $P$, but also on the parameters of $Q$.

In order to make VAEs work, it is essential to drive $Q$ to produce codes for $X$ that $P$ can reliably decode.

$$
E_{X\sim D}\left[E_{\epsilon\sim\mathcal{N}(0,I)}[\log P(X|z=\mu(X)+\Sigma^{1/2}(X)*\epsilon)]-\mathcal{D}\left[Q(z|X)\|P(z)\right]\right].
$$